In [443]:
import pandas as pd

In [444]:
df_loan = pd.read_csv(r'C:\Kamal\Machine Learing and AI\Course Sessions\Statistical Inference\Exploratory Data Analysis\Case_Study_1\loan.csv')

C:\Users\Kamalm\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (47) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [445]:
df_loan.shape

(39717, 111)

In [446]:
# Set right column names

df_loan = df_loan.rename(columns = {c : c.replace(' ','').replace('.','_') for c in df_loan.columns})
#df_loan = df_loan.rename(columns = {c : c.replace('.','_') for c in df_loan.columns})

print(df_loan.columns)

Index(['id', 'member_id', 'loan_amnt', 'funded_amnt', 'funded_amnt_inv',
       'term', 'int_rate', 'installment', 'grade', 'sub_grade',
       ...
       'num_tl_90g_dpd_24m', 'num_tl_op_past_12m', 'pct_tl_nvr_dlq',
       'percent_bc_gt_75', 'pub_rec_bankruptcies', 'tax_liens',
       'tot_hi_cred_lim', 'total_bal_ex_mort', 'total_bc_limit',
       'total_il_high_credit_limit'],
      dtype='object', length=111)


In [447]:
# Check potential unique value fields 
df_colunique = pd.Series({col: len(pd.unique(df_loan[col])) for col in df_loan}).reset_index()
df_colunique[df_colunique[0] == len(df_loan)]

# Remove the columns with all nulls ## potential unique 

,index,0
0,id,39717
1,member_id,39717
18,url,39717


In [448]:
# Strip all columns for leading and lagging spaces

df_loan = df_loan.applymap(lambda s : s.strip() if type(s) is str else s)

In [449]:
# Remove the columns with all nulls 

df_loan_null = df_loan.isnull().sum(axis = 0).reset_index()
columns_to_remove = df_loan_null[df_loan_null[0] == len(df_loan) ]

print(len(df_loan.columns))

df_loan.drop(columns_to_remove['index'] , axis = 1 , inplace = True)

df_loan.columns

111


Index(['id', 'member_id', 'loan_amnt', 'funded_amnt', 'funded_amnt_inv',
       'term', 'int_rate', 'installment', 'grade', 'sub_grade', 'emp_title',
       'emp_length', 'home_ownership', 'annual_inc', 'verification_status',
       'issue_d', 'loan_status', 'pymnt_plan', 'url', 'desc', 'purpose',
       'title', 'zip_code', 'addr_state', 'dti', 'delinq_2yrs',
       'earliest_cr_line', 'inq_last_6mths', 'mths_since_last_delinq',
       'mths_since_last_record', 'open_acc', 'pub_rec', 'revol_bal',
       'revol_util', 'total_acc', 'initial_list_status', 'out_prncp',
       'out_prncp_inv', 'total_pymnt', 'total_pymnt_inv', 'total_rec_prncp',
       'total_rec_int', 'total_rec_late_fee', 'recoveries',
       'collection_recovery_fee', 'last_pymnt_d', 'last_pymnt_amnt',
       'next_pymnt_d', 'last_credit_pull_d', 'collections_12_mths_ex_med',
       'policy_code', 'application_type', 'acc_now_delinq',
       'chargeoff_within_12_mths', 'delinq_amnt', 'pub_rec_bankruptcies',
       'tax_

In [450]:
print(len(df_loan.columns))

57


In [451]:
# Delete unwanted columns : less data 

# Count number of nulls

df_loan_null = df_loan.isnull().sum(axis = 0).reset_index()
df_loan_null['Percent'] = df_loan_null[0] / len(df_loan) *100
df_loan_null[df_loan_null.Percent > 0].sort_values(by = 'Percent' , ascending = False).head()

,index,0,Percent
47,next_pymnt_d,38577,97.129693
29,mths_since_last_record,36931,92.985372
28,mths_since_last_delinq,25682,64.662487
19,desc,12940,32.580507
10,emp_title,2459,6.191303


In [452]:
# Delete unwanted columns :  repetition and unwanted data 

# url has no additional information 

print(df_loan[(df_loan.id.astype(str) != df_loan.url.apply(lambda s : s[s.index('=') + 1:len(s)].strip() if '=' in s else s ))])
print(df_loan.url.apply(lambda s : s[0 : s.index('=')].strip() if '=' in s else s ).unique())

Empty DataFrame
Columns: [id, member_id, loan_amnt, funded_amnt, funded_amnt_inv, term, int_rate, installment, grade, sub_grade, emp_title, emp_length, home_ownership, annual_inc, verification_status, issue_d, loan_status, pymnt_plan, url, desc, purpose, title, zip_code, addr_state, dti, delinq_2yrs, earliest_cr_line, inq_last_6mths, mths_since_last_delinq, mths_since_last_record, open_acc, pub_rec, revol_bal, revol_util, total_acc, initial_list_status, out_prncp, out_prncp_inv, total_pymnt, total_pymnt_inv, total_rec_prncp, total_rec_int, total_rec_late_fee, recoveries, collection_recovery_fee, last_pymnt_d, last_pymnt_amnt, next_pymnt_d, last_credit_pull_d, collections_12_mths_ex_med, policy_code, application_type, acc_now_delinq, chargeoff_within_12_mths, delinq_amnt, pub_rec_bankruptcies, tax_liens]
Index: []

[0 rows x 57 columns]
['https://lendingclub.com/browse/loanDetail.action?loan_id']


In [453]:
# Remove string months from term 6 

df_loan.term = df_loan.term.apply(lambda s : s[0:s.index(' ')] if ' ' in s else s)
df_loan.term = pd.to_numeric(df_loan.term)
df_loan.term.head()

0    36
1    60
2    36
3    36
4    60
Name: term, dtype: int64

In [454]:
# Remove % sign from interest rate 

df_loan.int_rate = df_loan.int_rate.apply(lambda s : s[0:s.index('%')] if '%' in s else s)
df_loan.int_rate = pd.to_numeric(df_loan.int_rate)
df_loan.int_rate.head()

0    10.65
1    15.27
2    15.96
3    13.49
4    12.69
Name: int_rate, dtype: float64

In [456]:
df_loan.emp_length.apply(lambda s : str(s).replace('years' , '').replace(' ' , '').replace('year' , '') ).head()
#df_loan.emp_length = pd.to_numeric(df_loan.emp_length)

0    10+
1     <1
2    10+
3    10+
4      1
Name: emp_length, dtype: object

In [457]:
# Delete blank rows 
col_count = len(df_loan.columns)
df_loan[df_loan.isnull().sum(axis = 1) / col_count * 100  > 12 ]

(df_loan.isnull().sum(axis = 1)/ col_count * 100 ).max()

14.035087719298245

In [441]:
# Set the columns as date columns 

14.035087719298245

In [459]:
# Set the columns as numeric columns 

In [458]:
# Identify the number of rows where the loan is rejected 

